In [1]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

读入数据retail.csv，其中有以下注意事项：
（1）有不少的销售记录没有customerid，因此前后购买记录没法建立联系，需要删除（建议尝试：sum(records.customerid.isnull())）
（2）每一个customerid的数据类型是float65，不好处理，因此需要强制编程整数（建议尝试：type(records.customerid[0])）

In [12]:
records = pd.read_csv("retail.csv")
keepid=records.customerid.notnull()
records=records[keepid]
records['customerid'] = records['customerid'].map(int) 
records

,customerid,itemid,description,price
0,13085,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,6.95
1,13085,79323P,PINK CHERRY LIGHTS,6.75
2,13085,79323W,WHITE CHERRY LIGHTS,6.75
3,13085,22041,"RECORD FRAME 7"" SINGLE SIZE",2.10
4,13085,21232,STRAWBERRY CERAMIC TRINKET BOX,1.25
...,...,...,...,...
525456,17530,22271,FELTCRAFT DOLL ROSIE,2.95
525457,17530,22750,FELTCRAFT PRINCESS LOLA DOLL,3.75
525458,17530,22751,FELTCRAFT PRINCESS OLIVIA DOLL,3.75
525459,17530,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,3.75


创建一个关于itemid和description的字典

In [13]:
product_dictionary=dict(zip(records.itemid,records.description))
M=len(product_dictionary)
product_dictionary

{'85048': '15CM CHRISTMAS GLASS BALL 20 LIGHTS',
 '79323P': 'PINK CHERRY LIGHTS',
 '79323W': 'WHITE CHERRY LIGHTS',
 '22041': 'RECORD FRAME 7" SINGLE SIZE ',
 '21232': 'STRAWBERRY CERAMIC TRINKET BOX',
 '22064': 'PINK DOUGHNUT TRINKET POT ',
 '21871': 'SAVE THE PLANET MUG',
 '21523': 'DOORMAT FANCY FONT HOME SWEET HOME',
 '22350': 'ILLUSTRATED CAT BOWL ',
 '22349': 'DOG BOWL CHASING BALL DESIGN',
 '22195': 'LARGE HEART MEASURING SPOONS',
 '22353': 'LUNCH BOX WITH CUTLERY FAIRY CAKES ',
 '48173C': 'DOORMAT BLACK FLOCK ',
 '21755': 'LOVE BUILDING BLOCK WORD',
 '21754': 'HOME BUILDING BLOCK WORD',
 '84879': 'ASSORTED COLOUR BIRD ORNAMENT',
 '22119': 'PEACE WOODEN BLOCK LETTERS',
 '22142': 'CHRISTMAS CRAFT WHITE FAIRY ',
 '22296': 'HEART IVORY TRELLIS LARGE',
 '22295': 'HEART FILIGREE DOVE LARGE',
 '22109': 'FULL ENGLISH BREAKFAST PLATE',
 '22107': 'PIZZA PLATE IN BOX',
 '22194': 'BLACK DINER WALL CLOCK',
 '35004B': 'SET OF 3 BLACK FLYING DUCKS',
 '82582': 'AREA PATROLLED METAL SIGN',
 '21

为了演示方便，做一个非常粗糙但是可能很不合理的假设。
我们假设这个数据提供的，基于同一个customerid的购买记录是有顺序
关系的。那么，我们就可以把itermid看做是一个类似于文本的
序列数据，可以尝试使用word2vec模型。
因此，需要首先将所有的购买记录根据customerid分组

In [22]:
train_list = [] 
groups = records.groupby('customerid') 
for group in groups:
    temp_df = group[1] 
    my_records = list(temp_df['itemid']) 
    train_list.append(my_records)
train_list[0]

['TEST001',
 'TEST001',
 'TEST001',
 'TEST001',
 'TEST002',
 'TEST001',
 'TEST001',
 'TEST001',
 'TEST001',
 'ADJUST',
 '20682',
 '20679',
 '15056N',
 '15056BL',
 '15056P',
 '21524',
 '22692',
 '22660',
 '22687',
 '48173C',
 '22691',
 '48111',
 '22690',
 '21523',
 '48138',
 '22689',
 '22365',
 '48185',
 '22688',
 '48188',
 '48187',
 '22366',
 '20685',
 '21955',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'M',
 'D',
 'TEST001']

尝试建立一个100维的word2vec模型

In [23]:
DIM=100
model = Word2Vec(train_list, size=DIM, min_count=1)

首先获得所有不通的word（通过index2word），计算这些WORD的个数，获得他们的空间坐标Z

In [30]:
products=model.wv.index2word
M=len(products)
Z=model[products]
Z[0:3]

array([[ 0.15752213, -0.5165619 , -0.26788312, -0.16503944, -0.6424896 ,
         0.03931098,  0.24851932, -0.11454716,  0.55939555,  0.31503895,
         0.21066654,  0.2736492 , -0.01517976, -0.10214178,  1.2502097 ,
        -0.8733756 , -1.1400348 , -0.18991897,  1.0701243 , -0.4091238 ,
         0.1244534 ,  1.0565805 , -0.46885234, -0.10108585, -0.5915346 ,
        -0.82301813, -0.34875503, -0.07104318,  0.03979056, -0.43620247,
        -0.3220226 ,  0.00311274, -0.75405407,  0.8115999 , -0.22225317,
         0.03890958, -0.289688  , -0.23909158, -0.77360594,  0.45786974,
        -0.05267379, -0.13698891,  0.43345013, -0.36342636,  0.07727329,
         0.9221388 ,  0.12608965, -0.22981396, -1.009063  , -0.95041883,
        -0.39198932, -1.2381929 , -0.37512362, -0.42205396,  0.26581043,
        -0.25531316,  0.24952294, -0.35777044,  0.85006505, -0.1609628 ,
         0.13056388,  0.8485658 , -0.6912431 ,  0.13554278, -0.3120228 ,
        -0.1564961 ,  0.24611819,  0.4088816 , -0.1

每一个消费者可能购买多个商品。每个商品有一个空间坐标
然后，将多个空间坐标求均值，用于描述该消费者的空间位置

In [34]:
N=len(train_list)
X=np.zeros([N,DIM])
for i in range(N):
    my_records=train_list[i]
    my_state=model[my_records]
    X[i]=np.mean(my_state,0)

任意指定MyID号顾客，打印他的消费记录

In [35]:
MyID=20
my_records=train_list[MyID]
for i in range(len(my_records)): print(product_dictionary[my_records[i]])

POSTAGE
ROUND SNACK BOXES SET OF4 WOODLAND 
RED RETROSPOT PICNIC BAG
SET/20 WOODLAND PAPER NAPKINS 
SET/20 RED RETROSPOT PAPER NAPKINS 
RETROSPOT LARGE MILK JUG
LUNCH BAG RED RETROSPOT
RED RETROSPOT CUP
GREEN POLKADOT CUP
JUMBO BAG RED RETROSPOT
WOODLAND CHARLOTTE BAG
RED RETROSPOT CHARLOTTE BAG
LUNCH BAG WOODLAND
WOODLAND DESIGN  COTTON TOTE BAG
MUSHROOM BLUE HOT WATER BOTTLE
RED RETROSPOT PURSE 
RED SPOTTY TABLECLOTH 
RED RETROSPOT TRADITIONAL TEAPOT 
JUMBO BAG WOODLAND ANIMALS
LUNCH BAG CARS BLUE
RED RETROSPOT SHOPPER BAG
RETROSPOT CHILDRENS APRON
RED RETROSPOT CUP
RED POLKADOT PUDDING BOWL
RED POLKADOT BEAKER 
RED POLKADOT COFFEE  MUG
RED RETROSPOT TRADITIONAL TEAPOT 
RETROSPOT LARGE MILK JUG
RED RETROSPOT MUG
ROUND SNACK BOXES SET OF4 WOODLAND 
ROUND SNACK BOXES SET OF 4 SKULLS
SKULLS PARTY BAG + STICKER SET
RED RETROSPOT SHOPPER BAG
CHARLOTTE BAG SUKI DESIGN
SUKI  SHOULDER BAG
ROUND SNACK BOXES SET OF 4 FRUITS 
POSTAGE


计算该消费者的空间坐标位置与其他各个产品之间的距离位置，并寻找最近的产品

In [36]:
mystate=X[MyID]
similarity=Z-np.dot(np.ones([M,1]),mystate.reshape([1,DIM]))
similarity=np.mean(similarity**2,1)
TopProd=sorted(zip(similarity,products))
TopProd=TopProd[0][1]
TopProd=product_dictionary[TopProd]
TopProd

'RED RETROSPOT PURSE '